In [3]:
from langsmith import Client
import openai 

import os 
import sys 

sys.path.insert(0, '../..')

from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


from src.api.rag.retrieval_generation import (
    get_embeddings,
    retrieve_data,
    process_context,
    build_prompt,
    generate_answer,
    rag_pipeline,
)

/Users/eliekawerk/PycharmProjects/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Downlaod an example datapoint from Langsmith

In [4]:
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [5]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [6]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG', data_type=<DataType.kv: 'kv'>, id=UUID('aef32955-5a30-444c-bacf-4b3da59a8515'), created_at=datetime.datetime(2025, 10, 12, 18, 26, 55, 133747, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 12, 18, 26, 55, 133747, tzinfo=datetime.timezone.utc), example_count=33, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.7.1-arm64-arm-64bit', 'sdk_version': '0.4.34', 'runtime_version': '3.12.2', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': '0.3.79'}})

In [19]:
list(client.list_examples(dataset_name="rag-evaluation-dataset", limit=10))

[<class 'langsmith.schemas.Example'>(id=acde385d-7d0f-4772-95e5-4c137d1a2598, dataset_id=aef32955-5a30-444c-bacf-4b3da59a8515, link='https://smith.langchain.com/o/8e3d1487-3f47-4c1c-9cb0-18779336b64c/datasets/aef32955-5a30-444c-bacf-4b3da59a8515/e/acde385d-7d0f-4772-95e5-4c137d1a2598'),
 <class 'langsmith.schemas.Example'>(id=77b4020a-94d4-472d-8afc-6c01a948a961, dataset_id=aef32955-5a30-444c-bacf-4b3da59a8515, link='https://smith.langchain.com/o/8e3d1487-3f47-4c1c-9cb0-18779336b64c/datasets/aef32955-5a30-444c-bacf-4b3da59a8515/e/77b4020a-94d4-472d-8afc-6c01a948a961'),
 <class 'langsmith.schemas.Example'>(id=3a8b0e22-3fe6-4c05-96a7-ed10d0e5a91a, dataset_id=aef32955-5a30-444c-bacf-4b3da59a8515, link='https://smith.langchain.com/o/8e3d1487-3f47-4c1c-9cb0-18779336b64c/datasets/aef32955-5a30-444c-bacf-4b3da59a8515/e/3a8b0e22-3fe6-4c05-96a7-ed10d0e5a91a'),
 <class 'langsmith.schemas.Example'>(id=80778e60-5f01-47d0-a5b0-abcc770f7d91, dataset_id=aef32955-5a30-444c-bacf-4b3da59a8515, link='htt

In [14]:
list(client.list_examples(dataset_name="rag-evaluation-dataset"))[3].outputs

{'ground_truth': 'We do not stock fresh produce such as fruits or vegetables.',
 'reference_context_ids': [],
 'reference_descriptions': []}

In [15]:
list(client.list_examples(dataset_name="rag-evaluation-dataset"))[3].inputs

{'question': 'Do you have any fresh produce like fruits or vegetables?'}

In [31]:
example = list(client.list_examples(dataset_name="rag-evaluation-dataset", limit=10))

reference_input = example[5].inputs
reference_output = example[5].outputs

In [32]:
reference_input

{'question': 'Are there any wireless keyboard and mouse combos available?'}

In [33]:
reference_output

{'ground_truth': 'Yes, we offer a 2.4G Retro Computer Wireless Keyboard and Mouse Combo with quiet round keys and plug-and-play setup.',
 'reference_context_ids': ['B09PYFMTBF'],
 'reference_descriptions': ['KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision

### RAG Pipeline

In [12]:
qdrant_client_ = QdrantClient(url="http://localhost:6333")

In [13]:
rag_pipeline("Can you help me find a charger?", qdrant_client_=qdrant_client_)

{'answer': 'Sure! Here are some charger cable options available for you:\n\n1. iPhone Charger Cord Lightning Cables (3 pack, 3ft) - Apple MFi Certified, durable, fast charging and data transfer, compatible with iPhone 13/12/11 and many other models. (B0BYYLJRHT)\n\n2. 5 in 1 USB C to Multi Charging Cable (10ft) - Apple MFi Certified, supports Lightning, Type C, and Micro USB connectors, can charge three devices simultaneously, suitable for iPhone, Android, Samsung, Huawei, and more. (B0BFPZGYLD)\n\n3. Mixblu Charger Cable Replacement for Fitbit Inspire 3 (2 pack, 3.3ft) - Specifically for Fitbit Inspire 3, fast and stable charging. (B0BGDQLZD2)\n\n4. GREPHONE 2 Pack USB C to Lightning Cable (6ft) - Apple MFi Certified, fast charging, extra long cable, compatible with iPhone 13/12/11 and iPad models. (B0BV6PWVCG)\n\n5. MUXA 6 Pack iPhone Charger Colorful Nylon Lightning Cable (various lengths 3/3/6/6/10/10 ft) - Apple MFi Certified, durable, fast charging and data transfer, compatible w

### RAGAS Evaluation

In [20]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/var/folders/0b/khj15n_j3fb41qbm7n36lnsh0000gn/T/ipykernel_55557/168323522.py:4: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/0b/khj15n_j3fb41qbm7n36lnsh0000gn/T/ipykernel_55557/168323522.py:5: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [34]:
result = rag_pipeline(
    reference_input["question"],
    qdrant_client_
)

In [35]:
result

{'answer': 'Yes, there is a wireless keyboard and mouse combo available: the Wireless Keyboard and Mouse Combo (B09PYFMTBF). It features a 2.4G connection with a 2-in-1 nano USB receiver, silent click and retro round keycaps, a slim and compact design, ergonomic tilt, and compatibility with Windows computers and laptops.',
 'question': 'Are there any wireless keyboard and mouse combos available?',
 'retrieved_context_ids': ['B09PYFMTBF',
  'B09PRK49JH',
  'B0C9XFF3CT',
  'B0BF18F6R7',
  'B09LVX3XW2'],
 'retrieved_context': ['Wireless Keyboard and Mouse Combo, 2.4G Retro Computer Keyboard Mouse with Round Keys, Slim Quiet Keyboard Mouse with 2 in 1 Nano USB Receiver for for Windows, Laptop, PC, Notebook-Black Grey TRUE WIRELESS & PLUG AND PLAY: This wireless keyboard mouse set can keep you from the mess of the various cables on the desktop. The keyboard needs 2 AAA batteries and the mouse needs 1 AA battery. Just plug the receiver into the USB port and it quickly establishes a solid (up

In [39]:
async def ragas_faithfulness(run, example):
    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"]
    )
    scorer = Faithfulness(llm=ragas_llm)
    return await scorer.single_turn_ascore(sample)

In [40]:
await ragas_faithfulness(result, "")

0.8571428571428571

In [ ]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"]
    )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [42]:
await ragas_responce_relevancy(result, "")

np.float64(0.9356154996257272)

In [43]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
        retrieved_context_ids=run["retrieved_context_ids"],
        reference_context_ids=example["reference_context_ids"]
    )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [46]:
await ragas_context_precision_id_based(result, reference_output)

0.2

In [47]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [48]:
await ragas_context_recall_id_based(result, reference_output)

1.0